In [1]:
!nvidia-smi

Sun Mar 28 06:15:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install ISR

In [ ]:
!pip install tensorflow==2.3.0 

In [10]:
!rm -rf /content/sample_data
!rm -rf /content/images*
!cp -r "/content/drive/MyDrive/Colab Notebooks/Tianchi/CVPR2021 安全AI挑战者计划第六期：ImageNet无限制对抗攻击 赛道二/images-14.zip" /content/

!unzip -q /content/images-14.zip

# ISR

In [1]:
import numpy as np
import json
import os
import sys
import time
import pandas as pd
from PIL import Image
from tqdm import tqdm

from ISR.models import RDN, RRDN

import tensorflow as tf

In [2]:
# TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
#repo: https://github.com/idealo/image-super-resolution
#doc:  https://idealo.github.io/image-super-resolution/
    # model = RRDN(weights='gans')
    model = RDN(weights='psnr-small')
    # model = RDN(weights='noise-cancel')
    # model = RDN(weights='psnr-small')
    # model = RDN(weights='psnr-large')

Running on TPU  ['10.98.215.146:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.98.215.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.98.215.146:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
!rm -rf new_images
!mkdir new_images

In [4]:
def isr(img):
    imag_name = img.replace('images','new_images')

    img = Image.open(img)
    img = img.resize((200, 200))

    np_img = np.array(img)
    
    
    with strategy.scope():
        #model = model
        isr_img = model.predict(np_img)
        
        isr_img = Image.fromarray(isr_img)
        isr_img = isr_img.resize((500, 500), Image.ANTIALIAS)
        isr_img.save(imag_name, quality=75)

In [5]:
image_list = []
for name in range(0,5000):
    image_list.append('images/{}.jpg'.format(name))
image_list[-1]

'images/4999.jpg'

In [6]:
for img in tqdm(image_list):
    isr(img)

  0%|          | 0/5000 [00:00<?, ?it/s]

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
100%|██████████| 5000/5000 [1:08:46<00:00,  1.21it/s]


In [7]:
!rm -rf new_images.zip
!zip -r -q new_images.zip new_images/*.jpg
!cp -r /content/new_images.zip '/content/drive/MyDrive/Colab Notebooks/Tianchi/CVPR2021 安全AI挑战者计划第六期：ImageNet无限制对抗攻击 赛道二/'